In [1]:
#%reset -f
%matplotlib inline
import os
import sys
import pandas as pd
import numpy as np
import feather
import time
import pogs as pogs



Loaded POGS GPU library.

> Setting pogs.SolverCPU=None


> Setting pogs.ElasticNetSolverCPU=None
> Use pogs.ElasticNetSolverGPU(args...) and re-run setup.py




### Import Data Frame and create raw X and y arrays

In [2]:
t0 = time.time()
if not os.path.exists("../R/ipums_200-2015.csv"):
    print("Please download https://drive.google.com/file/d/0By-R0tLVMSykckNocHZPeEN1VU0/view?usp=sharing")
if not os.path.exists("ipums.feather"):
    !R -f ../R/ipums_prep.R 
df = feather.read_dataframe("ipums.feather")
#df = pd.read_csv("../R/data.csv")
t1 = time.time()
print("Time to read data via feather: %r" % (t1-t0))

Please download https://drive.google.com/file/d/0By-R0tLVMSykckNocHZPeEN1VU0/view?usp=sharing
Time to read data via feather: 7.2249815464019775


In [3]:
target = df.columns[-1] ## last column is the response
cols = [c for c in df.columns if c != target]

In [4]:
X = np.array(df.ix[:,cols])
y = df[target].values
print(X.shape)
print(y.shape)

(55776, 9732)
(55776,)


### H2O AI GLM using the GPU Pointers

In [5]:
sourceDev = 0
gpu=True
nThreads = 2
nGPUs = 2
intercept = 1
lambda_min_ratio = 1e-4
nLambdas = 100
nAlphas = 1

H = (int)(0.8*X.shape[0])

trainX = X[:H,:]
trainY = y[:H]

validX = X[H:,:]
validY = y[H:]

## TODO: compute these in C++ (CPU or GPU)
lambda_max0 = max(abs(trainX.T.dot(trainY)))
sdTrainY = np.sqrt(np.var(trainY))
meanTrainY = np.mean(trainY)
mTrain = trainX.shape[0]
mValid = validX.shape[0]
n = trainX.shape[1]
print(mTrain)
print(mValid)
print(n)

44620
11156
9732


In [6]:
Solver = pogs.ElasticNetSolverGPU if gpu else pogs.ElasticNetSolverCPU
enet = Solver(nThreads, nGPUs, 'r', intercept, lambda_min_ratio, nLambdas, nAlphas)

In [7]:
a,b,c,d = enet.upload_data(sourceDev, trainX, trainY, validX, validY)

/home/arno/.pyenv/versions/3.6.0/envs/h2oai/lib/python3.6/site-packages/pogs/solvers/elastic_net_base.py:19: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  mValid = validX.shape[0] if validX != None else 0
/home/arno/.pyenv/versions/3.6.0/envs/h2oai/lib/python3.6/site-packages/pogs/solvers/elastic_net_base.py:27: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  C = cptr(validX,c_double) if validX != None else A
/home/arno/.pyenv/versions/3.6.0/envs/h2oai/lib/python3.6/site-packages/pogs/solvers/elastic_net_base.py:28: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  D = cptr(validY,c_double) if validX != None else B


c_void_p(1112589467648)
c_void_p(1116063402496)
c_void_p(1116064448512)
c_void_p(1116934766592)


In [ ]:
t0 = time.time()
enet.fit(sourceDev, mTrain, n, mValid, lambda_max0, sdTrainY, meanTrainY, a, b, c, d)
t1 = time.time()
print("Time to train H2O AI GLM: %r" % (t1-t0))